In [0]:
import pandas as pd
import numpy as np
import os
import ast

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#https://stackoverflow.com/a/48438170
%cd "drive/My Drive"
!pwd

In [0]:
files = os.listdir('patdata')

In [0]:
df = pd.DataFrame(columns = ['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'new_label'])

In [0]:
label_map = {'Sleep stage W': 0, 'Sleep stage 1': 1, 'Sleep stage 2': 2, 'Sleep stage 3': 3, 'Sleep stage 4': 4, 'Sleep stage R': 5}
def literal_return(val):
    try:
        return ast.literal_eval(val)
    except ValueError:
        return (val)
for file in files:
  print(file)
  path='patdata/'+file
  df_m=pd.read_csv(path)
  df_m=pd.read_csv(path,dtype= {"val1": object, "val2": object, "val3": object})
  df_m=df_m.rename(columns={"val1":'EEG Fpz-Cz',"val2":"EEG Pz-Oz","val3":"EOG horizontal","val4":"EMG submental"})
  df_m['new_label']=[label_map[x] for x in df_m['labels']]
  df_m=df_m[['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'new_label']]
  df_m['EEG Fpz-Cz'] = df_m['EEG Fpz-Cz'].apply(literal_return)
  df_m['EEG Pz-Oz'] = df_m['EEG Pz-Oz'].apply(literal_return)
  #df_m['EEG Fpz-Cz']=[ast.literal_eval(x) for x in df_m['EEG Fpz-Cz']]
  #df_m['EEG Pz-Oz']=[ast.literal_eval(x) for x in df_m['EEG Pz-Oz']]
  df_m['EOG horizontal']=df_m['EOG horizontal'].apply(literal_return)
  df=df.append(df_m)


In [0]:
pred_data11 = list(df['EEG Pz-Oz'])
for i in range(len(pred_data11)):
  if len(pred_data11[i]) != 300:
    pred_data11[i].extend([0.0]*(300-len(pred_data11[i])))
pred_data11 = np.asarray(pred_data11)
pred_data11.shape

In [0]:
from numpy import newaxis
pred_data11 = pred_data11[:, newaxis, :]
pred_data11.shape

In [0]:
pred_data12 = list(df['EEG Fpz-Cz'])
for i in range(len(pred_data12)):
  if len(pred_data12[i]) != 300:
    pred_data12[i].extend([0.0]*(300-len(pred_data12[i])))
pred_data12 = np.asarray(pred_data12)
pred_data12 = pred_data12[:, newaxis, :]
pred_data12.shape

In [0]:
pred_data13 = list(df['EOG horizontal'])
for i in range(len(pred_data13)):
  if len(pred_data13[i]) != 300:
    pred_data13[i].extend([0.0]*(300-len(pred_data13[i])))
pred_data13 = np.asarray(pred_data13)
pred_data13 = pred_data13[:, newaxis, :]
pred_data13.shape

In [0]:
pred_data1 = np.concatenate((np.concatenate((pred_data11, pred_data12),axis = 1), pred_data13),axis = 1)
pred_data1.shape

In [0]:
target_data1 = np.array(df['new_label'].values, dtype=int)
target_data1.shape

In [0]:
from sklearn.model_selection import train_test_split
import torch

x_train, x_test, y_train, y_test = train_test_split(pred_data1, target_data1, test_size=0.3, random_state=0)

In [0]:
#x_train.tofile('x_train.dat')
#x_test.tofile('x_test.dat')
y_train.tofile('y_train.dat', sep=',')
y_test.tofile('y_test.dat', sep=',')

In [0]:
import numpy as np
import pandas as pd
from scipy import sparse
import torch
from torch.utils.data import TensorDataset, Dataset
import itertools
import collections
from collections import OrderedDict
from tensorflow import feature_column

In [0]:
x_train.shape

In [0]:
pred_train = torch.tensor(x_train.astype(np.float32))
#pred_train = torch.from_numpy(x_train)
target_train = torch.tensor(y_train)

pred_test = torch.tensor(x_test.astype(np.float32))
target_test = torch.tensor(y_test)

train_data = TensorDataset(pred_train, target_train)
test_data = TensorDataset(pred_test, target_test)

## Utility Functions

In [0]:
import os
import time
import numpy as np
import torch


class AverageMeter(object):
	"""Computes and stores the average and current value"""

	def __init__(self):
		self.reset()

	def reset(self):
		self.val = 0
		self.avg = 0
		self.sum = 0
		self.count = 0

	def update(self, val, n=1):
		self.val = val
		self.sum += val * n
		self.count += n
		self.avg = self.sum / self.count


def compute_batch_accuracy(output, target):
	"""Computes the accuracy for a batch"""
	with torch.no_grad():

		batch_size = target.size(0)
		_, pred = output.max(1)
		correct = pred.eq(target).sum()

		return correct * 100.0 / batch_size


def train(model, device, data_loader, criterion, optimizer, epoch, print_freq=10):
	batch_time = AverageMeter()
	data_time = AverageMeter()
	losses = AverageMeter()
	accuracy = AverageMeter()

	model.train()

	end = time.time()
	for i, (input, target) in enumerate(data_loader):
		# measure data loading time
		data_time.update(time.time() - end)

		if isinstance(input, tuple):
			input = tuple([e.to(device) if type(e) == torch.Tensor else e for e in input])
		else:
			input = input.to(device)
		target = target.to(device)

		optimizer.zero_grad()
		output = model(input)
		loss = criterion(output, target)
		assert not np.isnan(loss.item()), 'Model diverged with loss = NaN'

		loss.backward()
		optimizer.step()

		# measure elapsed time
		batch_time.update(time.time() - end)
		end = time.time()

		losses.update(loss.item(), target.size(0))
		accuracy.update(compute_batch_accuracy(output, target).item(), target.size(0))

		if i % print_freq == 0:
			print('Epoch: [{0}][{1}/{2}]\t'
				  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
				  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
				  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
				  'Accuracy {acc.val:.3f} ({acc.avg:.3f})'.format(
				epoch, i, len(data_loader), batch_time=batch_time,
				data_time=data_time, loss=losses, acc=accuracy))

	return losses.avg, accuracy.avg


def evaluate(model, device, data_loader, criterion, print_freq=10):
	batch_time = AverageMeter()
	losses = AverageMeter()
	accuracy = AverageMeter()

	results = []

	model.eval()

	with torch.no_grad():
		end = time.time()
		for i, (input, target) in enumerate(data_loader):

			if isinstance(input, tuple):
				input = tuple([e.to(device) if type(e) == torch.Tensor else e for e in input])
			else:
				input = input.to(device)
			target = target.to(device)

			output = model(input)
			loss = criterion(output, target)

			# measure elapsed time
			batch_time.update(time.time() - end)
			end = time.time()

			losses.update(loss.item(), target.size(0))
			accuracy.update(compute_batch_accuracy(output, target).item(), target.size(0))

			y_true = target.detach().to('cpu').numpy().tolist()
			y_pred = output.detach().to('cpu').max(1)[1].numpy().tolist()
			results.extend(list(zip(y_true, y_pred)))

			if i % print_freq == 0:
				print('Test: [{0}/{1}]\t'
					  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
					  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
					  'Accuracy {acc.val:.3f} ({acc.avg:.3f})'.format(
					i, len(data_loader), batch_time=batch_time, loss=losses, acc=accuracy))

	return losses.avg, accuracy.avg, results


class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

## Models

In [0]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
from torch.autograd import Variable

class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv1d(3,6,5)  #unimproved model <---- change in_channels from 1 to 3
        self.conv2 = nn.Conv1d(6,16,5)  #unimproved model
        self.rel = nn.ReLU()  #unimproved model
        self.max_pool = nn.MaxPool1d(2)  #unimproved model
        self.layer1 = nn.Linear(72*16, 128)  #unimproved model
        self.layer2 = nn.Linear(128,6)  #unimproved model
        self.dropout = nn.Dropout(0.2)  #improved model

    def forward(self, x):
        # x = self.max_pool(self.rel(self.conv1(x)))  #unimproved model
        # x = self.max_pool(self.rel(self.conv2(x)))  #unimproved model
        # x = x.view(-1, 656)  #unimproved model
        # x = self.rel(self.layer1(x))  #unimproved model
        # x = self.layer2(x)  #unimproved model
        x = self.max_pool(self.rel(self.dropout(self.conv1(x))))  #improved model
        x = self.max_pool(self.rel(self.dropout(self.conv2(x))))  #improved model
        x = x.view(-1, 72*16)  #improved model
        x = self.rel(self.dropout(self.layer1(x)))  #improved model
        x = self.layer2(x)  #improved model
        return x

In [0]:
import torch.nn as nn
class MyRNN(nn.Module):
    def __init__(self):
        super(MyRNN, self).__init__()
        self.rnn = nn.GRU(input_size=300, hidden_size=16,
                          num_layers=3, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(in_features=16, out_features=6)
        self.rel = nn.ReLU()

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.rel(x[:, -1, :])
        # x = self.fc(x[:, -1, :])
        x = self.fc(x)
        return x

## Plots

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
# TODO: You can use other packages if you want, e.g., Numpy, Scikit-learn, etc.


def plot_learning_curves(train_losses, valid_losses, train_accuracies, valid_accuracies):
	# TODO: Make plots for loss curves and accuracy curves.
	# TODO: You do not have to return the plots.
	# TODO: You can save plots as files by codes here or an interactive way according to your preference.
    fig1 = plt.figure()
    plt.plot(train_losses, label = "Training Loss")
    plt.plot(valid_losses, label = "Validation Loss")
    minposs = valid_losses.index(min(valid_losses))+1 
    plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')
    plt.legend(loc="upper left")
    fig1.suptitle('Loss Curve', fontsize=20)
    plt.xlabel('epoch')
    plt.ylabel('Loss')
    plt.savefig('loss_curves.png')
    plt.show()
    plt.clf()
    fig2 = plt.figure()
    plt.plot(train_accuracies, label = "Training Accuracy")
    plt.plot(valid_accuracies, label = "Validation Accuracy")
    plt.legend(loc="upper left")
    fig2.suptitle('Accuracy Curve', fontsize=20)
    plt.xlabel('epoch')
    plt.ylabel('Accuracy')
    plt.savefig('accuracy_curves.png')
    plt.show()


def plot_confusion_matrix(results, class_names):
	# TODO: Make a confusion matrix plot.
	# TODO: You do not have to return the plots.
	# TODO: You can save plots as files by codes here or an interactive way according to your preference.
    y_true = [i[0] for i in results]
    y_pred = [i[1] for i in results]
    C = confusion_matrix(y_true, y_pred)
    C1 = np.around(C/C.astype(np.float).sum(axis=1).reshape(C.shape[0],1), decimals=2)
    #C2 = pd.DataFrame(C1, range(C.shape[0]), range(C.shape[0]))
    #sn.heatmap(C2, annot=True, xticklabels=class_names, yticklabels=class_names)
    fig, ax = plt.subplots()
    im = ax.imshow(C1)
    ax.set_xticks(np.arange(len(class_names)))
    ax.set_xticklabels(class_names)
    ax.set_yticks(np.arange(len(class_names)))
    ax.set_yticklabels(class_names)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    for i in range(C1.shape[0]):
        for j in range(C1.shape[0]):
            text = ax.text(j, i, C1[i, j], ha="center", va="center", color="w")
    ax.set_title("Normalized confusion matrix")
    fig.tight_layout()
    plt.savefig('normalized_confusion_matrix.png')

## Training

In [0]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

NUM_EPOCHS = 40
BATCH_SIZE = 32
USE_CUDA = False  # Set 'True' if you want to use GPU
NUM_WORKERS = 0  # Number of threads used by DataLoader. You can adjust this according to your machine spec.

model = MyRNN()
#model = RNN()
#model = MyRCNN()

device = torch.device("cuda" if USE_CUDA and torch.cuda.is_available() else "cpu")
torch.manual_seed(1)
if device.type == "cuda":
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False
    
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
valid_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion.to(device)

best_val_acc = 0.0
train_losses, train_accuracies = [], []
valid_losses, valid_accuracies = [], []

early_stopping = EarlyStopping(patience=20, verbose=True)
for epoch in range(NUM_EPOCHS):
	train_loss, train_accuracy = train(model, device, train_loader, criterion, optimizer, epoch)
	valid_loss, valid_accuracy, valid_results = evaluate(model, device, valid_loader, criterion)

	train_losses.append(train_loss)
	valid_losses.append(valid_loss)

	train_accuracies.append(train_accuracy)
	valid_accuracies.append(valid_accuracy)

	is_best = valid_accuracy > best_val_acc  # let's keep the model that has the best accuracy, but you can also use another metric.
	if is_best:
		best_val_acc = valid_accuracy
		#torch.save(model, os.path.join(PATH_OUTPUT, save_file))
		best_model = model
	early_stopping(valid_loss, model)
	if early_stopping.early_stop:
		print("early Stopping")
		break

In [0]:
plot_learning_curves(train_losses, valid_losses, train_accuracies, valid_accuracies)
test_loss, test_accuracy, test_results = evaluate(best_model, device, valid_loader, criterion)

class_names = ['Sleep stage W', 'Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R']#, 'Sleep stage ?']
#class_names = ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R']
plot_confusion_matrix(test_results, class_names)